In [1]:
import importlib, numpy as np
import laser
from tqdm import tqdm
from rich import print
import pandas as pd
from math import gcd, isclose
import pickle
import plotly.graph_objs as go
from functools import reduce
import plotly.express as px
importlib.reload(laser)


las = laser.Laser()

In [3]:
def float_gcd(a: float, b: float, scale: float = 1e6) -> float:
    """Approximate GCD for floats using integer scaling."""
    a_int, b_int = round(a * scale), round(b * scale)
    return gcd(a_int, b_int) / scale

def multi_float_gcd(freqs, scale: float = 1e6) -> float:
    """GCD of multiple float frequencies."""
    return reduce(lambda x, y: float_gcd(x, y, scale), freqs)

def common_cycle_time(*hz_values, scale: float = 1e6) -> float:
    """Returns the time (in seconds) until all input frequencies align in phase."""
    if len(hz_values) == 1 and hasattr(hz_values[0], '__iter__'):
        hz_values = list(hz_values[0])

    hz_values = [f for f in hz_values if not isclose(f, 0)]
    if not hz_values:
        return 0.0

    g = multi_float_gcd(hz_values, scale)
    return 1 / g



In [ ]:
# make chords

min_cycle_time = 0.015

start = round(1/min_cycle_time)
stop = 1200
hzs = np.arange(start, stop, .25)

chords = []
for i in tqdm(hzs):
    arr = [i]
    for j in hzs:
        if j not in arr and common_cycle_time(*arr, j) < min_cycle_time:
            arr.append(j)
    chords.append(sorted(arr))


with open('chords', 'wb') as fp:
    pickle.dump(chords, fp)

In [10]:
def max_amplitude(self: laser.Laser, arr: list, seconds=1):
    print(arr)
    color = [i for i in arr if i[0] in ['R', 'G', 'B']]
    arr = [i for i in arr if i not in ['R', 'G', 'B']]

    start_amp = [i[2] for i in arr]
    new_amp = [i[2] for i in self.softmax_normalize_by_label(arr)]

    steps = round(seconds * 159)
    t_vals = np.linspace(0, 1, steps)
    blend_vals = 0.5 * (1 - np.cos(np.pi * t_vals))  # cosine interpolation weights

    # cosine-tempered transitions
    trans = [
        start + (end - start) * blend_vals
        for start, end in zip(start_amp, new_amp)
    ]

    for amps in zip(*trans):
        new_arr = [[i[0], i[1], amp] for amp, i in zip(amps, arr)] + color
        self.send(new_arr, first=False)
    return new_arr

def transition(self: laser.Laser, start: list, end: list, color: list, seconds=2):
    different_indexes = [n for n, (a, b) in enumerate(zip(start, end)) if not all([i==j for i, j in zip(a, b)])]
    start_amp = [i[2] for i in self.softmax_normalize_by_label(start)]
    mid = [i[2] for i in self.softmax_normalize_by_label([i if n not in different_indexes else [i[0], i[1], i[2] * -100000] for n, i in enumerate(start)])]
    new_amp = [i[2] for i in self.softmax_normalize_by_label(end)]

    # cosine-tempered transitions
    cos = np.cos(np.pi * np.linspace(0, 1, round(seconds * 159/2)))
    trans1 = [start + (end - start) * 0.5 * (1 - cos) for start, end in zip(start_amp, mid)]
    trans2 = [start + (end - start) * 0.5 * (1 - cos) for start, end in zip(mid,   new_amp)]
    full_transitions = np.array([list(a) + list(b) for a, b in zip(trans1, trans2)]).T

    for i in full_transitions:
        tmp = [[j[0], j[1], i[n]] for n, j in enumerate(end)]
        self.send(tmp + color, first=False)
    return tmp

amp = 19
color = [['R', amp], ['B', amp], ['G', amp]]
arr1 = las.random_chord(6, 1)
new = las.random_chord(6, 1)
arr2 = [i for i in arr1]
arr2[0] = new[0]

arr = transition(las, arr1, arr2, color)
las.show(arr, amp, 2, first=False)

las.off()

In [19]:
las.off()

In [ ]:
print(arr)
arr = max_amplitude(las, arr)

print(arr)
new_tone = las.random_chord(1, 1)[0]
old_index = np.random.choice(list(range(len(arr)-3)))
old_tone = arr.pop(old_index)
print(arr)
arr = transition(las, arr, old_tone, new_tone)

arr = max_amplitude(las, arr)
print(arr)
las.off()

In [ ]:
arr = las.random_chord()
amp = 19
las.show(arr + [['R', amp], ['B', amp], ['G', amp]], seconds=3)

In [60]:
with open('superGoodOnes', 'rb') as fp:
    previous = pickle.load(fp)

with open('superGoodOnes', 'wb') as fp:
    pickle.dump(previous + [arr], fp)

In [7]:
with open('goodOnes', 'rb') as fp:
    previous = pickle.load(fp)

with open('goodOnes', 'wb') as fp:
    pickle.dump(previous + [arr], fp)

In [3]:
with open('badOnes', 'rb') as fp:
    previous = pickle.load(fp)

with open('badOnes', 'wb') as fp:
    pickle.dump(previous + [arr], fp)

In [17]:
types = {}
with open('badOnes', 'rb') as fp:
    types['bad'] = pickle.load(fp)
with open('goodOnes', 'rb') as fp:
    types['good'] = pickle.load(fp)
count = 0

In [18]:

las.show_many(types['good'], 10, seconds=2)

KeyboardInterrupt: 